# Statistics - Models
Date : 10/08/2020

Load the csv gathering the models and their results.
The aim is to show the evolution of the accuracy and the parameters

In [1]:
import torch

In [2]:
# useful imports
from IPython.display import SVG, Audio, display
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
import librosa
%matplotlib inline
import os
from matplotlib import pyplot as plt

# voicemap imports
from voicemap.datasets import LibriSpeech, SpeakersInTheWild, CommonVoice, TCOF, ClassConcatDataset, SpectrogramDataset, DatasetFolder 
from config import PATH, DATA_PATH, LOG_PATH

## (Optional) Save a model

In [13]:
old_model_name = "../models/model=resnet__dim=1__lr=0.001__weight_decay=0.01__momentum=0.9__filters=128__batch_size=1500__n_seconds=3.0__use_standardized=True__spectrogram=True__n_t=0__T=None__n_f=0__F=None__num_samples=65970__num_classes=2199.pt"
new_model_name = "30-std_001-2.pt"

In [14]:
%%bash -s "$old_model_name" "$new_model_name"
cp $1 ../saved_models/$2

## (Optional) Append the model into the 'models' log file

In [15]:
def appendModel(old_name, new_name="Untitled"):
    old_name = old_name.split("/")[-1][:-3]
    df = pd.read_csv(LOG_PATH + '/models.csv')
    params = old_name.split("__")
    i = len(df)
    df.loc[i] = np.NaN
    df.loc[i, 'model_name'] = new_name
    for param in params:
        param_name, param_value = param.split("=")
        df.loc[i, param_name] = param_value
    return df
df = appendModel(old_model_name, new_model_name)
df.to_csv(f"{LOG_PATH}/models.csv", index=False)

## Load data

In [16]:
df_read = pd.read_csv(f"{LOG_PATH}/models.csv")
df_read

,model_name,details,model,dim,lr,weight_decay,momentum,filters,batch_size,n_seconds,...,window_hop,n_t,T,n_f,F,num_samples,num_classes,datasets,val_loss,accuracy
0,gaixoa.pt,NaN,resnet,1.0,0.000100,0.01,0.9,128.0,32.0,3.0,...,NaN,0.0,None,0.0,None,130475.0,1291.0,LS,0.1200,0.6580
1,babybabyMario.pt,NaN,resnet,1.0,0.000100,0.01,0.9,128.0,1500.0,3.0,...,NaN,0.0,None,0.0,None,133376.0,1291.0,LS,0.2380,0.7300
2,luigi_000001.pt,with commonvoice,resnet,1.0,0.000001,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,263978.0,3045.0,LSC,0.4136,0.8437
3,peach_00001.pt,with TCOF,resnet,1.0,0.000010,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,465597.0,3699.0,LSCT,1.0800,0.5800
4,std_00001.pt,with 15-standardized,resnet,1.0,0.000010,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,45255.0,3017.0,LCT,1.2108,0.2200
5,30-std_001.pt,NaN,resnet,1.0,0.001000,0.01,0.9,128.0,1500.0,3.0,...,NaN,0.0,None,0.0,None,65970.0,2199.0,NaN,NaN,NaN
6,30-std_001-2.pt,NaN,resnet,1.0,0.001000,0.01,0.9,128.0,1500.0,3.0,...,NaN,0.0,None,0.0,None,65970.0,2199.0,NaN,NaN,NaN


## 1. Evolution of accuracy & loss

Avant l'ajout de TCOF (et donc des voix d'enfants) on obtient un excellent résultat. Le modèle n'était cependant pas performant avec des voix d'enfants.
L'ajout de TCOF a fait largement augmenté la perte et baissé la précision globale. L'amélioration sur les locuteurs enfants n'a pas encore été testée.

#### Affichage des logs des entraînements des modèles

In [12]:
model_log = pd.read_csv("../logs/model=resnet__dim=1__lr=0.001__weight_decay=0.01__momentum=0.9__filters=128__batch_size=1500__n_seconds=3.0__use_standardized=True__spectrogram=True__n_t=0__T=None__n_f=0__F=None__num_samples=65970__num_classes=2199.csv")
model_log

,epoch,accuracy,common_voice_test_auc,common_voice_test_eer,librispeech_dev_clean_auc,librispeech_dev_clean_eer,loss,lr,sitw_eval_auc,sitw_eval_eer,tcof_dev_auc,tcof_dev_eer,train_accuracy,train_loss,val_accuracy,val_loss
0,1,0.000000,0.937422,0.167240,0.714295,0.350785,18.375948,0.001,0.502540,0.509839,0.631078,0.357143,0.002958,7.580870,0.002729,7.661680
1,2,0.000000,0.974835,0.055000,0.771382,0.298429,17.807340,0.001,0.539569,0.480403,0.643321,0.357143,0.015167,7.001735,0.009550,7.209467
2,3,0.000000,0.983561,0.033624,0.826174,0.243455,17.334913,0.001,0.556197,0.465565,0.653835,0.357143,0.035625,6.561897,0.019858,6.906831
3,4,0.000000,0.985616,0.027940,0.849505,0.227749,16.918063,0.001,0.574727,0.440000,0.672010,0.357143,0.075458,6.146649,0.030014,6.659097
4,5,0.000000,0.986977,0.024818,0.863115,0.208038,16.527479,0.001,0.587077,0.467097,0.660105,0.357143,0.124958,5.798525,0.043353,6.460541
5,6,0.000000,0.987378,0.024338,0.877148,0.197557,16.161323,0.001,0.600798,0.440000,0.660746,0.357143,0.193333,5.419819,0.053054,6.293913
6,7,0.000000,0.989192,0.020014,0.887405,0.181961,15.806123,0.001,0.616415,0.380000,0.645426,0.357143,0.269708,5.083042,0.067152,6.153966
7,8,0.000000,0.987938,0.020014,0.887016,0.195082,15.462049,0.001,0.606171,0.401855,0.655260,0.357143,0.370083,4.735455,0.081249,6.010506
8,9,0.000000,0.987244,0.023537,0.887871,0.172775,15.116852,0.001,0.631561,0.370565,0.617995,0.460035,0.421333,4.534471,0.087009,5.956091
9,10,0.000000,0.987564,0.020655,0.892544,0.168675,14.809524,0.001,0.625281,0.395403,0.632503,0.425276,0.531583,4.146367,0.101410,5.811567


In [4]:
import optuna
study = optuna.load_study(study_name='study-01', storage=f"sqlite:///{LOG_PATH}/optimization.db")
study.get_trials()

[FrozenTrial(number=0, value=None, datetime_start=datetime.datetime(2020, 8, 14, 16, 35, 8, 463488), datetime_complete=datetime.datetime(2020, 8, 14, 16, 35, 13, 311753), params={'batch_size': 128, 'epochs': 3, 'filters': 128, 'lr': 0.1, 'train_datasets_letters': 'LCT', 'window_hop': 0.02418908593342099, 'window_length': 0.09291820094735158}, distributions={'batch_size': CategoricalDistribution(choices=(128, 32, 1)), 'epochs': IntUniformDistribution(high=15, low=1, step=1), 'filters': CategoricalDistribution(choices=(64, 128)), 'lr': CategoricalDistribution(choices=(0.1, 0.01)), 'train_datasets_letters': CategoricalDistribution(choices=('LCT', 'CT')), 'window_hop': UniformDistribution(high=0.5, low=0.01), 'window_length': UniformDistribution(high=0.5, low=0.01)}, user_attrs={}, system_attrs={'fail_reason': "Trial 0 failed because of the following error: RuntimeError('Given groups=1, weight of size [128, 744, 7], expected input[128, 161, 124] to have 744 channels, but got 161 channels i